In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!nvidia-smi
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, TextStreamer
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)
!pip install huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.tom

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `third` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
import torch
from transformers import AutoModelForCausalLM

base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [6]:
!wget -O task_vector_french_alpaca.pt "https://polybox.ethz.ch/index.php/s/j9uKW83LXfTFFPM/download"
!wget -O task_vector_arabic_alpaca.pt "https://polybox.ethz.ch/index.php/s/eHNpPPahdYFaAPu/download"

#!wget -O french_1000.txt "https://polybox.ethz.ch/index.php/s/GhaB7tmz50NwLvd/download"
#!wget -O japanese_1000.txt "https://polybox.ethz.ch/index.php/s/OWRbornV830Q052/download"
#!wget -O arabic_1000.txt "https://polybox.ethz.ch/index.php/s/Ee8uK9H2Rq6QSds/download"
!wget -O french_1000.txt "https://polybox.ethz.ch/index.php/s/6JBqNQ0YpjwSy5H/download"
!wget -O japanese_1000.txt "https://polybox.ethz.ch/index.php/s/IGRkHgDILSbleGB/download"
!wget -O arabic_1000.txt "https://polybox.ethz.ch/index.php/s/TV1s9Dkd3JW191x/download"
#!wget -O arabic_1000.txt "https://polybox.ethz.ch/index.php/s/JjmSB4vhi8wfgUK/download"

--2025-03-07 16:53:55--  https://polybox.ethz.ch/index.php/s/j9uKW83LXfTFFPM/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5591628774 (5.2G) [application/octet-stream]
Saving to: ‘task_vector_french_alpaca.pt’

task_vector_french_ 100%[===================>]   5.21G  30.0MB/s    in 3m 2s   

2025-03-07 16:56:58 (29.3 MB/s) - ‘task_vector_french_alpaca.pt’ saved [5591628774/5591628774]

--2025-03-07 16:56:58--  https://polybox.ethz.ch/index.php/s/eHNpPPahdYFaAPu/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5591628774 (5.2G) [application/octet-stream]
Saving to: ‘task_vector_arabic_alpaca.pt’

task_vector_arabic_ 100%[===================>]   5.21G  28.6MB/s    in 3m 10s

In [24]:
!wget -O french_1000.txt "https://polybox.ethz.ch/index.php/s/6JBqNQ0YpjwSy5H/download"
!wget -O japanese_1000.txt "https://polybox.ethz.ch/index.php/s/IGRkHgDILSbleGB/download"
!wget -O arabic_1000.txt "https://polybox.ethz.ch/index.php/s/TV1s9Dkd3JW191x/download"

--2025-03-07 17:14:48--  https://polybox.ethz.ch/index.php/s/6JBqNQ0YpjwSy5H/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 397292 (388K) [text/plain]
Saving to: ‘french_1000.txt’

french_1000.txt     100%[===================>] 387.98K   671KB/s    in 0.6s    

2025-03-07 17:14:49 (671 KB/s) - ‘french_1000.txt’ saved [397292/397292]

--2025-03-07 17:14:49--  https://polybox.ethz.ch/index.php/s/IGRkHgDILSbleGB/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3432371 (3.3M) [text/plain]
Saving to: ‘japanese_1000.txt’

japanese_1000.txt   100%[===================>]   3.27M  3.60MB/s    in 0.9s    

2025-03-07 17:14:50 (3.60 MB/s) - ‘japanese_1000.txt’ saved [3432371/3432371]

In [7]:
task_vector_french = torch.load("task_vector_french_alpaca.pt")
task_vector_arabic = torch.load("task_vector_arabic_alpaca.pt")

<ipython-input-7-22974905b710>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  task_vector_french = torch.load("task_vector_french_alpaca.pt")
<ipython-input-7-22974905b710

In [8]:
import numpy as np
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_eos_token=True, padding_side='left')
class MAPAlgorithm:
    def __init__(self, task_vectors, text_samples):
        self.task_vectors = task_vectors
        self.n_tasks = len(self.task_vectors)
        #if len(text_samples) != self.n_tasks:
            #raise ValueError("Number of metric models must match the number of tasks.")
        self.text_samples = text_samples

    def sample_coefficients(self, k=3):
        #coeffs = np.random.rand(k, self.n_tasks)
        #coeffs = coeffs / coeffs.sum(axis=1, keepdims=True)
        #return torch.tensor(coeffs, dtype=torch.float32)
        values = np.arange(0.05, 0.9, 0.1)  # Values between 0 and <0.6 with 0.1 steps
        coeffs = np.array(np.meshgrid(values, values)).T.reshape(-1, 2)
        return torch.tensor(coeffs, dtype=torch.float32)


    # for one metric but all coeffs
    def compute_metrics(self, coeffs, index):
        # coeffs: list of coefficient vectors
        metrics = []
        text_sample = self.text_samples[index]
        for c in coeffs:
            task_v1 = self.task_vectors[0]
            task_v2 = self.task_vectors[1]

            original_state_dict = {name: param.clone() for name, param in base_model.named_parameters()}

            for param, delta in zip(base_model.parameters(), task_v1):
              param.data.add_(torch.tensor(c[0], dtype=param.dtype, device=param.device) * delta)

            for param, delta in zip(base_model.parameters(), task_v2):
              param.data.add_(torch.tensor(c[1], dtype=param.dtype, device=param.device) * delta)

            metric = compute_perplexity(base_model, tokenizer, text_sample)
            metrics.append(metric)

            '''for param, delta in zip(base_model.parameters(), task_v1):
              param.data.sub_(torch.tensor(c[0], dtype=param.dtype, device=param.device) * delta)

            for param, delta in zip(base_model.parameters(), task_v2):
              param.data.sub_(torch.tensor(c[1], dtype=param.dtype, device=param.device) * delta)'''

            for name, param in base_model.named_parameters():
              param.data.copy_(original_state_dict[name])
        return metrics

    def construct_z_vector(self, c):
        n = len(c)
        z = [1] + list(c)
        z += [0.5 * c[i]**2 for i in range(n)]
        z += [c[i] * c[j] for i in range(n) for j in range(i+1, n)]
        return np.array(z)

    def construct_Z_matrix(self, coeffs):
        return np.array([self.construct_z_vector(c) for c in coeffs])

    def fit_quadratic_model(self, coeffs, n_metrics):
        n_tasks = n_metrics - 1
        surrogate_models = []
        for metric_idx in range(n_metrics):
            print("Now Metric: ", metric_idx)
            y = np.array(self.compute_metrics(coeffs,metric_idx))
            #minimize ||Zx - y||^2 for the given coeffs and target vector y.
            Z = self.construct_Z_matrix(coeffs)
            x, _, _, _ = np.linalg.lstsq(Z, y, rcond=None)

            e = x[0]
            b = x[1:n_tasks+1]
            A = np.zeros((n_tasks, n_tasks))
            A[np.arange(n_tasks), np.arange(n_tasks)] = x[n_tasks+1:2*n_tasks+1]

            off_diagonal_elements = x[2*n_tasks+1:]
            idx = 0
            for i in range(n_tasks):
                for j in range(i+1, n_tasks):
                    A[i, j] = off_diagonal_elements[idx]
                    A[j, i] = off_diagonal_elements[idx]
                    idx += 1

            surrogate_models.append((A, b, e))

        return surrogate_models

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [26]:
def compute_perplexity(model, tokenizer, texts, max_samples=300):
    """Compute perplexity over a dataset sample."""
    model.eval()
    total_loss = 0
    total_tokens = 0

    for text in texts[:max_samples]:
        if len(text.strip()) == 0:
            continue
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to("cuda")

        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            total_loss += loss.item() * inputs["input_ids"].shape[1]  # Weight loss by token count
            total_tokens += inputs["input_ids"].shape[1]  # Track total tokens

    avg_perplexity = torch.exp(torch.tensor(total_loss / total_tokens))  # Normalize by tokens
    return avg_perplexity.item()

In [25]:
with open("french_1000.txt", "r", encoding="utf-8") as f:
    french_text = [line.strip() for line in f.readlines()]

print(f"French dataset loaded successfully! Total lines: {len(french_text)}")

with open("arabic_1000.txt", "r", encoding="utf-8") as f:
    arabic_text = [line.strip() for line in f.readlines()]

print(f"Arabic dataset loaded successfully! Total lines: {len(arabic_text)}")

with open("japanese_1000.txt", "r", encoding="utf-8") as f:
    japanese_text = [line.strip() for line in f.readlines()]

print(f"Japanese dataset loaded successfully! Total lines: {len(japanese_text)}")

French dataset loaded successfully! Total lines: 719
Arabic dataset loaded successfully! Total lines: 1000
Japanese dataset loaded successfully! Total lines: 31757


In [ ]:
task_vectors=[task_vector_french, task_vector_arabic]
text_samples = [french_text, arabic_text, japanese_text]
map_algorithm = MAPAlgorithm(task_vectors, text_samples)

In [ ]:
coeffs = map_algorithm.sample_coefficients(k=4)
surrogate_models = map_algorithm.fit_quadratic_model(coeffs, len(text_samples))

Now Metric:  0


<ipython-input-34-da447c37d891>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  param.data.add_(torch.tensor(c[0], dtype=param.dtype, device=param.device) * delta)
<ipython-input-34-da447c37d891>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  param.data.add_(torch.tensor(c[1], dtype=param.dtype, device=param.device) * delta)


Now Metric:  1
Now Metric:  2


In [ ]:
import json
import numpy as np
# save the surrogate models

def save_surrogate_models(surrogate_models, file_path):
    surrogate_models_to_save = []

    for A, b, e in surrogate_models:
        surrogate_models_to_save.append({
            "A": A.tolist(),
            "b": b.tolist(),
            "e": e.tolist()
        })
    with open(file_path, 'w') as f:
        json.dump(surrogate_models_to_save, f, indent=4)

    print(f"Surrogate models saved to {file_path}")

save_surrogate_models(surrogate_models, "surrogate_models.json")

Surrogate models saved to surrogate_models.json


In [29]:
#french base:
ppl_score = compute_perplexity(base_model, tokenizer, french_text)
print(f"Perplexity: {ppl_score}")

Perplexity: 9.940449714660645


In [19]:
#french taskv:
original_state_dict = {name: param.clone() for name, param in base_model.named_parameters()}
for param, delta in zip(base_model.parameters(), task_vector_french):
  param.data.add_(delta)
ppl_score = compute_perplexity(base_model, tokenizer, french_text)
for name, param in base_model.named_parameters():
  param.data.copy_(original_state_dict[name])
print(f"Perplexity: {ppl_score}")

Perplexity: 8.172281265258789


In [30]:
#arabic basae:
ppl_score = compute_perplexity(base_model, tokenizer, arabic_text)
print(f"Perplexity: {ppl_score}")

Perplexity: 16.053407669067383


In [27]:
#arabic:
original_state_dict = {name: param.clone() for name, param in base_model.named_parameters()}
for param, delta in zip(base_model.parameters(), task_vector_arabic):
  param.data.add_(delta)
ppl_score = compute_perplexity(base_model, tokenizer, arabic_text)
for name, param in base_model.named_parameters():
  param.data.copy_(original_state_dict[name])
print(f"Perplexity: {ppl_score}")

Perplexity: 12.8008451461792


In [23]:
#japanese base:
ppl_score = compute_perplexity(base_model, tokenizer, japanese_text)
print(f"Perplexity: {ppl_score}")

Perplexity: 53.093345642089844


Now, we how well the Pareto optimal solution of PMM performs:

In [28]:
#c=[0.34201116, 0.21936332]
#c = [0.74206186, 0.83823027]
c = [0.68389244, 0.4378204]
original_state_dict = {name: param.clone() for name, param in base_model.named_parameters()}
for param, delta in zip(base_model.parameters(), task_vector_french):
  param.data.add_(torch.tensor(c[0], dtype=param.dtype, device=param.device) * delta)
for param, delta in zip(base_model.parameters(), task_vector_arabic):
  param.data.add_(torch.tensor(c[1], dtype=param.dtype, device=param.device) * delta)
ppl_score_french = compute_perplexity(base_model, tokenizer, french_text)
ppl_score_arabic = compute_perplexity(base_model, tokenizer, arabic_text)
ppl_score_japanese = compute_perplexity(base_model, tokenizer, japanese_text)
for name, param in base_model.named_parameters():
  param.data.copy_(original_state_dict[name])
print(f"Perplexity French: {ppl_score_french}")
print(f"Perplexity Arabic: {ppl_score_arabic}")
print(f"Perplexity Japanese: {ppl_score_japanese}")

Perplexity French: 9.897022247314453
Perplexity Arabic: 15.07198429107666
Perplexity Japanese: 50.94782638549805
